In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import random
from tqdm import trange

/home/nbuser/anaconda3_501/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_501/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data2 = pd.read_csv("white.csv")

In [3]:
data2 = pd.concat([data2.iloc[:,0:11], pd.get_dummies(data2["y"])], axis = 1)

In [4]:
def minimini(x, size, seed = 0): # 미니배치 구현
    np.random.seed(seed)
    m = x.shape[0]
    mini = []
    per = list(np.random.permutation(m))
    shuffle_x = x.iloc[per, :]
    num = math.floor(m/size)
    for k in range(num):
        mini.append(shuffle_x.iloc[k*size:(k+1)*size, : ])
    if m % size != 0:
        mini.append(shuffle_x.iloc[num*size:m, :])
    return mini

### 1번
sigmoid 함수 이용, GradientDescentOptimizer

In [49]:
#1
dim = 11
learn_rate = 0.001
train_epoch = 101
batch_size = 128

X = tf.placeholder(tf.float32, shape = [None, 11])
Y = tf.placeholder(tf.float32, shape = [None, 7])

w0 = tf.Variable(tf.zeros(shape = [dim, 256]))
w1 = tf.Variable(tf.zeros(shape = [256, 128]))
w2 = tf.Variable(tf.zeros(shape = [128, 7]))

b0 = tf.Variable(tf.zeros(shape = [256]))
b1 = tf.Variable(tf.zeros(shape = [128]))
b2 = tf.Variable(tf.zeros(shape = [7]))

def classifier(x):
    c1 = tf.nn.relu(tf.add(tf.matmul(x, w0), b0))
    c2 = tf.nn.relu(tf.add(tf.matmul(c1, w1), b1))
    return tf.nn.sigmoid(tf.add(tf.matmul(c2, w2), b2))

y = classifier(X)
cost = -tf.reduce_sum(Y*tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(train_epoch):
    avg_cost = 0
    a = minimini(data2, batch_size, seed = i)
    gr = list(np.random.permutation(math.floor(data2.shape[0]/batch_size)))
    for step in range(math.floor(data2.shape[0]/batch_size)):
            temp = a[gr[step]]
            xs = temp.iloc[:, 0:11]
            ys = temp.iloc[:, 11:]
            _, c = sess.run([optimizer, cost], feed_dict = {X: xs, Y: ys})
            avg_cost += c/math.floor(data2.shape[0]/batch_size)
    if i % 10  == 0:
        print("epoch"+str(i), avg_cost)
sess.close()

epoch0 69.93737009951943
epoch10 12.007026622169892
epoch20 6.8725834896689975
epoch30 4.8468804673144685
epoch40 3.7609592174228865
epoch50 3.0623210856789047
epoch60 2.6125997210803784
epoch70 2.271466562622472
epoch80 2.012270667051014
epoch90 1.8129157800423468
epoch100 1.6458870015646279


### 2번

Layer 확대와 Node 수 변경, sigmoid 함수 이용, Adamoptimizer

In [48]:
#2
dim = 11
learn_rate = 0.001
train_epoch = 101
batch_size = 128

X = tf.placeholder(tf.float32, shape = [None, 11])
Y = tf.placeholder(tf.float32, shape = [None, 7])

w0 = tf.Variable(tf.zeros(shape = [dim, 512]))
w1 = tf.Variable(tf.zeros(shape = [512, 256]))
w2 = tf.Variable(tf.zeros(shape = [256, 128]))
w3 = tf.Variable(tf.zeros(shape = [128, 7]))

b0 = tf.Variable(tf.zeros(shape = [512]))
b1 = tf.Variable(tf.zeros(shape = [256]))
b2 = tf.Variable(tf.zeros(shape = [128]))
b3 = tf.Variable(tf.zeros(shape = [7]))

def classifier(x):
    c1 = tf.nn.relu(tf.add(tf.matmul(x, w0), b0))
    c2 = tf.nn.relu(tf.add(tf.matmul(c1, w1), b1))
    c3 = tf.nn.relu(tf.add(tf.matmul(c2, w2), b2))
    return tf.nn.sigmoid(tf.add(tf.matmul(c3, w3), b3))

y = classifier(X)
cost = -tf.reduce_sum(Y*tf.log(y))
optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(train_epoch):
    avg_cost = 0
    a = minimini(data2, batch_size, seed = i)
    gr = list(np.random.permutation(math.floor(data2.shape[0]/batch_size)))
    for step in range(math.floor(data2.shape[0]/batch_size)):
            temp = a[gr[step]]
            xs = temp.iloc[:, 0:11]
            ys = temp.iloc[:, 11:]
            _, c = sess.run([optimizer, cost], feed_dict = {X: xs, Y: ys})
            avg_cost += c/math.floor(data2.shape[0]/batch_size)
    if i % 10  == 0:
        print("epoch"+str(i), avg_cost)
sess.close()

epoch0 87.5692028246428
epoch10 67.00544457686576
epoch20 51.70519858912419
epoch30 40.350440778230364
epoch40 31.870448765001793
epoch50 25.459972682752106
epoch60 20.558636012830227
epoch70 16.749345428065247
epoch80 13.752781943271033
epoch90 11.368091156608184
epoch100 9.448351508692694


### 3번
Weight 초기화하는 Xavier 알고리즘, Layer 확대와 Node 수 변경, Adamoptimizer, relu 함수 이용

In [51]:
#3
dim = 11
learn_rate = 0.001
train_epoch = 31
batch_size = 128

X = tf.placeholder(tf.float32, shape = [None, 11])
Y = tf.placeholder(tf.float32, shape = [None, 7])

def xavier(size):
    std = 1. / tf.sqrt(size[0]/2.)
    return tf.random_normal(shape = size, stddev = std)

w0 = tf.Variable(xavier(size = [dim, 512]))
w1 = tf.Variable(xavier(size = [512, 256]))
w2 = tf.Variable(xavier(size = [256, 128]))
w3 = tf.Variable(xavier(size = [128, 7]))

b0 = tf.Variable(tf.zeros(shape = [512]))
b1 = tf.Variable(tf.zeros(shape = [256]))
b2 = tf.Variable(tf.zeros(shape = [128]))
b3 = tf.Variable(tf.zeros(shape = [7]))

def classifier(x):
    c1 = tf.nn.relu(tf.add(tf.matmul(x, w0), b0))
    c2 = tf.nn.relu(tf.add(tf.matmul(c1, w1), b1))
    c3 = tf.nn.relu(tf.add(tf.matmul(c2, w2), b2))
    return tf.nn.relu(tf.add(tf.matmul(c3, w3), b3))

y = classifier(X)
cost = -tf.reduce_sum(Y*tf.log(y + 1e-6))
optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(train_epoch):
    avg_cost = 0
    a = minimini(data2, batch_size, seed = i)
    gr = list(np.random.permutation(math.floor(data2.shape[0]/batch_size)))
    for step in range(math.floor(data2.shape[0]/batch_size)):
            temp = a[gr[step]]
            xs = temp.iloc[:, 0:11]
            ys = temp.iloc[:, 11:]
            _, c = sess.run([optimizer, cost], feed_dict = {X: xs, Y: ys})
            avg_cost += c/math.floor(data2.shape[0]/batch_size)
    if i % 10  == 0:
        print("epoch"+str(i), avg_cost)
sess.close()

epoch0 204.29310648064862
epoch10 -920.9188874897204
epoch20 -1175.0350422106292
epoch30 -1306.5669973273025


### 4번
Dropout, sigmoid 함수 이용, GradientDescentOptimizer, Layer 확대와 Node 수 변경

In [54]:
#4
dim = 11
learn_rate = 0.001
train_epoch = 101
batch_size = 128

X = tf.placeholder(tf.float32, shape = [None, 11])
Y = tf.placeholder(tf.float32, shape = [None, 7])
keep_prob = tf.placeholder(tf.float32)

w0 = tf.Variable(tf.zeros(shape = [dim, 512]))
w1 = tf.Variable(tf.zeros(shape = [512, 256]))
w2 = tf.Variable(tf.zeros(shape = [256, 128]))
w3 = tf.Variable(tf.zeros(shape = [128, 7]))

b0 = tf.Variable(tf.zeros(shape = [512]))
b1 = tf.Variable(tf.zeros(shape = [256]))
b2 = tf.Variable(tf.zeros(shape = [128]))
b3 = tf.Variable(tf.zeros(shape = [7]))


def classifier(x, keep_prob):
    c1 = tf.nn.relu(tf.add(tf.matmul(x, w0), b0))
    c2 = tf.nn.relu(tf.add(tf.matmul(c1, w1), b1))
    c3 = tf.nn.relu(tf.add(tf.matmul(c2, w2), b2))
    c_drop = tf.nn.dropout(c3, keep_prob)
    return tf.nn.sigmoid(tf.add(tf.matmul(c_drop, w3), b3))

y = classifier(X, keep_prob)
cost = -tf.reduce_sum(Y*tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(train_epoch):
    avg_cost = 0
    a = minimini(data2, batch_size, seed = i)
    gr = list(np.random.permutation(math.floor(data2.shape[0]/batch_size)))
    for step in range(math.floor(data2.shape[0]/batch_size)):
            temp = a[gr[step]]
            xs = temp.iloc[:, 0:11]
            ys = temp.iloc[:, 11:]
            _, c = sess.run([optimizer, cost], feed_dict = {X: xs, Y: ys, keep_prob: 0.7})
            avg_cost += c/math.floor(data2.shape[0]/batch_size)
    if i % 10  == 0:
        print("epoch"+str(i), avg_cost)
sess.close()

epoch0 69.93737009951943
epoch10 12.007026622169892
epoch20 6.8725834896689975
epoch30 4.8468804673144685
epoch40 3.7609592174228865
epoch50 3.0623210856789047
epoch60 2.6125997210803784
epoch70 2.271466562622472
epoch80 2.012270667051014
epoch90 1.8129157800423468
epoch100 1.6458870015646279


### 5번
Dropout, relu 함수 이용, Layer 확대와 Node 수 변경, Adamoptimizer, Weight 초기화하는 Xavier 알고리즘 이용

In [7]:
#5
dim = 11
learn_rate = 0.01
train_epoch = 101
batch_size = 128

X = tf.placeholder(tf.float32, shape = [None, 11])
Y = tf.placeholder(tf.float32, shape = [None, 7])
keep_prob = tf.placeholder(tf.float32)

def xavier(size):
    std = 1. / tf.sqrt(size[0]/2.)
    return tf.random_normal(shape = size, stddev = std)

w0 = tf.Variable(tf.zeros(shape = [dim, 256]))
w1 = tf.Variable(tf.zeros(shape = [256, 128]))
w2 = tf.Variable(tf.zeros(shape = [128, 7]))

b0 = tf.Variable(tf.zeros(shape = [256]))
b1 = tf.Variable(tf.zeros(shape = [128]))
b2 = tf.Variable(tf.zeros(shape = [7]))

def classifier(x, keep_prob):
    c1 = tf.nn.elu(tf.add(tf.matmul(x, w0), b0))
    c2 = tf.nn.elu(tf.add(tf.matmul(c1, w1), b1))
    c_drop = tf.nn.dropout(c2, keep_prob)
    return tf.nn.elu(tf.add(tf.matmul(c_drop, w2), b2))

y = classifier(X, keep_prob)

cost = -tf.reduce_sum(Y*tf.log(y + 1e-5))
optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(train_epoch):
    avg_cost = 0
    a = minimini(data2, batch_size, seed = i)
    gr = list(np.random.permutation(math.floor(data2.shape[0]/batch_size)))
    for step in range(math.floor(data2.shape[0]/batch_size)):
            temp = a[gr[step]]
            xs = temp.iloc[:, 0:11]
            ys = temp.iloc[:, 11:]
            _, c = sess.run([optimizer, cost], feed_dict = {X: xs, Y: ys, keep_prob: 0.1})
            avg_cost += c/math.floor(data2.shape[0]/batch_size)
    if i % 10  == 0:
        print("epoch"+str(i), avg_cost)
sess.close()

epoch0 426.54243388928876
epoch10 342.0379253186678
epoch20 317.96462450529395
epoch30 294.3061186137951
epoch40 272.4664772435238
epoch50 252.35262419048112
epoch60 233.80257817318565
epoch70 216.48261301141034
epoch80 200.14309290835732
epoch90 184.57526076467417
epoch100 169.67199385793583


### 결론

1. 자비어 친구를 적용하면 어느정도 처음 로스가 줄어든다? -> 실험이 좀 더 필요
2. relu로 너무 많이 쌓으면 안좋은거 같기도 하다 가끔 nan 나옴 ㅠㅠ
3. 미니배치의 필요성... 안그러면 학습이 너무 오래걸린다
4. relu dying problem이 너무 많이 나서 tf.clip_by_value(y_conv,1e-10,1.0) 이런식으로 최소 최대 값을 잡았다 3번과 5번 비교해보길 -> relu에서 elu로 바꿔버림
5. Adam은 GD과 수렴속도가 달라서 약간 learning rate의 조절이 필요하다